## VaR Self Learning Project

This project is just a self made way to better understand and comprehend Value at Risk or VaR. VaR is a measure of minimum loss for a porfolio / security. It is given as a probability related to the potential minimum loss, in a given time frame. IE: a portfolio can have a monthly 5% VaR of 5 million USD. This would mean that there is a 5% chance in a given month that the minimum loss will be at least 5 million USD.

There are 3 major ways to compute VaR. The Monte Carlo simulation method, historic simulation and parametric approach. I will cover each approach and their assumptions / methodology one at a time.

# Parametric Method

This method of calculating VaR is also called the variance-covariance method. It assumes that the distributions for the risk factors are normal. The main two things required are return expectation and standard deviation. This is a simple and straightforward approach to finding VaR however it is sensitive to return expectation, E(R), and standard deviation, σ. It is also difficult to compute if the portfolio contains options, since options would threaten the normality. 

Lets take a three position portfolio containing SPY, AAPL, VCRB. Each security will have an equal weighting. 

In [1]:
#Importing libraries
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
# Step 1: Download data
tickers = ['SPY', 'AAPL', 'VCRB']
data = yf.download(tickers, start='2024-01-01', end='2024-12-31')['Adj Close']
print("Step 1: Adjusted Close Prices")
print(data.head())

[*********************100%***********************]  3 of 3 completed

Step 1: Adjusted Close Prices
Ticker            AAPL         SPY       VCRB
Date                                         
2024-01-02  184.734970  466.663940  73.874931
2024-01-03  183.351746  462.852814  73.989067
2024-01-04  181.023163  461.361969  73.505684
2024-01-05  180.296707  461.993896  73.313858
2024-01-08  184.655365  468.589264  73.697502


In [3]:
returns = data.pct_change().dropna()
print("Step 2: Daily Returns")
print(returns.head())

Step 2: Daily Returns
Ticker          AAPL       SPY      VCRB
Date                                    
2024-01-03 -0.007488 -0.008167  0.001545
2024-01-04 -0.012700 -0.003221 -0.006533
2024-01-05 -0.004013  0.001370 -0.002610
2024-01-08  0.024175  0.014276  0.005233
2024-01-09 -0.002263 -0.001517 -0.001666


In [4]:
# Step 3: Calculate metrics for parametric VaR
weights = np.array([1/3, 1/3, 1/3])  # Equal weighting
mean_returns = returns.mean()  # Expected returns
cov_matrix = returns.cov()  # Covariance matrix
print("Step 3: Mean Returns (Expected Return for Each Security)")
print(mean_returns)
print("Covariance Matrix")
print(cov_matrix)

Step 3: Mean Returns (Expected Return for Each Security)
Ticker
AAPL    0.001345
SPY     0.000958
VCRB    0.000118
dtype: float64
Covariance Matrix
Ticker      AAPL       SPY      VCRB
Ticker                              
AAPL    0.000200  0.000060  0.000007
SPY     0.000060  0.000063  0.000003
VCRB    0.000007  0.000003  0.000011


In [5]:
# Portfolio metrics
expected_return = np.dot(weights, mean_returns)  # Portfolio expected return
portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))  # Portfolio std deviation
print("Step 3 (continued): Portfolio Metrics")
print(f"Expected Portfolio Return: {expected_return:.4f}")
print(f"Portfolio Standard Deviation: {portfolio_std:.4f}\n")


Step 3 (continued): Portfolio Metrics
Expected Portfolio Return: 0.0008
Portfolio Standard Deviation: 0.0068



In [6]:
# Step 4: Compute parametric VaR
confidence_level = 0.95
z_score = 1.645  # Z-score for 95% confidence level
VaR = z_score * portfolio_std - expected_return
print("Step 4: Parametric VaR Calculation")
print(f"Parametric VaR (95% confidence): {VaR:.4f}\n")

Step 4: Parametric VaR Calculation
Parametric VaR (95% confidence): 0.0104



Lets assume this portfolio value is 5 million USD. This means that there is a 5 percent chance there will be a minimum loss of at least 51,500 or 1.03% of the portfolio on a given day. However we can also adjust this so it is on a monthly basis instead.

In [7]:
# Monthly adjustments
monthly_expected_return = expected_return * 20
monthly_portfolio_std = portfolio_std * np.sqrt(20)

print("Monthly Portfolio Metrics:")
print(f"Monthly Expected Return: {monthly_expected_return:.4f}")
print(f"Monthly Standard Deviation: {monthly_portfolio_std:.4f}")

# Recalculate Monthly VaR
monthly_VaR = z_score * monthly_portfolio_std - monthly_expected_return
print(f"Parametric VaR (95% confidence, Monthly): {monthly_VaR:.4f}")

Monthly Portfolio Metrics:
Monthly Expected Return: 0.0161
Monthly Standard Deviation: 0.0304
Parametric VaR (95% confidence, Monthly): 0.0338


This means that during a month, with 20 trading days, there is a 5 percent chance of a minimum loss of 3.36% or 168,000

# Historical Simulation Approach

We will be using the same portfolio under the historical approach with the same assumptions. A big factor that can affect VaR will using historical simulation is if any major events occured. If something major occured then the data no longer becomes representitive. Major events could include a major contraction in the market, a war breaking out, elections and other macrofactors. Due to this the more data, the longer the period is, the higher the chance of bad data. 

In [8]:
# Step 1: Calculate portfolio returns
# Multiply individual returns by weights and sum for each day
portfolio_returns = returns.dot(weights)
print(portfolio_returns)

Date
2024-01-03   -0.004703
2024-01-04   -0.007485
2024-01-05   -0.001751
2024-01-08    0.014561
2024-01-09   -0.001815
                ...   
2024-12-23    0.002012
2024-12-24    0.007760
2024-12-26    0.001588
2024-12-27   -0.008429
2024-12-30   -0.006682
Length: 250, dtype: float64


In [9]:
# Step 2: Sort portfolio returns
sorted_returns = portfolio_returns.sort_values()
print(sorted_returns)

Date
2024-08-05   -0.025976
2024-12-18   -0.019189
2024-07-24   -0.017790
2024-09-03   -0.014239
2024-04-15   -0.013592
                ...   
2024-01-08    0.014561
2024-04-11    0.016272
2024-09-19    0.017623
2024-05-03    0.025900
2024-06-11    0.026366
Length: 250, dtype: float64


In [10]:
# Step 3: Calculate historical VaR
confidence_level = 0.95
var_percentile = int((1 - confidence_level) * len(sorted_returns))
historical_VaR = -sorted_returns.iloc[var_percentile]
# Output results
print("Step 3: Historical VaR Calculation")
print(f"Historical VaR (95% confidence): {historical_VaR:.4%}")

Step 3: Historical VaR Calculation
Historical VaR (95% confidence): 1.0973%


This means that using the historical simulation method our 5% VaR for the portfolio is 1.0973%. Or that there is a 5% chance of losing at least 54,865 in a single trading day. Lets compute the monthly VaR now

In [11]:
# Step 1: Calculate portfolio returns (daily)
portfolio_returns = returns.dot(weights)
print(portfolio_returns)


Date
2024-01-03   -0.004703
2024-01-04   -0.007485
2024-01-05   -0.001751
2024-01-08    0.014561
2024-01-09   -0.001815
                ...   
2024-12-23    0.002012
2024-12-24    0.007760
2024-12-26    0.001588
2024-12-27   -0.008429
2024-12-30   -0.006682
Length: 250, dtype: float64


In [12]:
# Step 2: Resample daily returns to monthly returns by summing daily returns for each month
monthly_returns = portfolio_returns.resample('M').sum()
print(monthly_returns)


Date
2024-01-31    0.006744
2024-02-29    0.006921
2024-03-31   -0.002860
2024-04-30   -0.022901
2024-05-31    0.064659
2024-06-30    0.046532
2024-07-31    0.030690
2024-08-31    0.024428
2024-09-30    0.018043
2024-10-31   -0.020079
2024-11-30    0.040032
2024-12-31    0.009523
Freq: M, dtype: float64


In [13]:
# Step 3: Sort the monthly returns in ascending order (to find the worst outcomes)
sorted_monthly_returns = monthly_returns.sort_values()
print(sorted_monthly_returns)


Date
2024-04-30   -0.022901
2024-10-31   -0.020079
2024-03-31   -0.002860
2024-01-31    0.006744
2024-02-29    0.006921
2024-12-31    0.009523
2024-09-30    0.018043
2024-08-31    0.024428
2024-07-31    0.030690
2024-11-30    0.040032
2024-06-30    0.046532
2024-05-31    0.064659
dtype: float64


In [14]:
# Step 4: Calculate the 5th percentile (for 95% confidence level)
confidence_level = 0.95
var_percentile = int((1 - confidence_level) * len(sorted_monthly_returns))
historical_monthly_VaR = -sorted_monthly_returns.iloc[var_percentile]

# Output the result
print(f"Monthly Historical VaR (95% confidence): {historical_monthly_VaR:.4%}")

Monthly Historical VaR (95% confidence): 2.2901%


# Monte Carlo Simulation Method

In [15]:
# Step 1: Calculate daily returns
returns = data.pct_change().dropna()
returns

Ticker,AAPL,SPY,VCRB
Date,,,
2024-01-03,-0.007488,-0.008167,0.001545
2024-01-04,-0.012700,-0.003221,-0.006533
2024-01-05,-0.004013,0.001370,-0.002610
2024-01-08,0.024175,0.014276,0.005233
2024-01-09,-0.002263,-0.001517,-0.001666
...,...,...,...
2024-12-23,0.003065,0.005988,-0.003018
2024-12-24,0.011478,0.011115,0.000688
2024-12-26,0.003176,0.000067,0.001521


In [16]:
# Step 2: Calculate mean returns and covariance matrix
mean_returns = returns.mean()
cov_matrix = returns.cov()
print(mean_returns)
print(cov_matrix)

Ticker
AAPL    0.001345
SPY     0.000958
VCRB    0.000118
dtype: float64
Ticker      AAPL       SPY      VCRB
Ticker                              
AAPL    0.000200  0.000060  0.000007
SPY     0.000060  0.000063  0.000003
VCRB    0.000007  0.000003  0.000011


In [17]:
# Step 3: Define simulation parameters
num_simulations = 50000  # Number of Monte Carlo simulations
num_days = 1  # 1 trading day for daily VaR
confidence_level = 0.95

# Step 4: Simulate portfolio returns using Monte Carlo method
simulated_returns = np.zeros(num_simulations)

for i in range(num_simulations):
    # Generate random returns based on the mean and covariance matrix
    daily_simulated_returns = np.random.multivariate_normal(mean_returns, cov_matrix, num_days)
    # Sum up the daily returns to get the daily return for this simulation
    simulated_returns[i] = np.sum(daily_simulated_returns @ weights)

In [18]:
# Step 5: Sort the simulated returns and calculate the VaR at the 5th percentile
sorted_simulated_returns = np.sort(simulated_returns)
var_percentile_index = int((1 - confidence_level) * num_simulations)
monte_carlo_var = -sorted_simulated_returns[var_percentile_index]

# Step 6: Print the results
print(f"Step 3: Monte Carlo Simulation VaR Calculation")
print(f"Monte Carlo VaR (95% confidence) for 1 day: {monte_carlo_var:.4%}")

Step 3: Monte Carlo Simulation VaR Calculation
Monte Carlo VaR (95% confidence) for 1 day: 1.0357%


This means that using the monte carlo simulation method our 5% VaR for the portfolio is 1.0357%. Or that there is a 5% chance of losing at least 51,785 in a single trading day. 

Now lets find the monthly VaR instead. 

In [19]:
# Step 1: Scale the daily returns to monthly by multiplying by sqrt(20)
num_days_in_month = 20  # Assuming 20 trading days in a month
monthly_simulated_returns = simulated_returns * np.sqrt(num_days_in_month)

# Step 2: Sort the monthly simulated returns and calculate the VaR at the 5th percentile
sorted_monthly_simulated_returns = np.sort(monthly_simulated_returns)
var_percentile_index_monthly = int((1 - confidence_level) * num_simulations)
monte_carlo_monthly_var = -sorted_monthly_simulated_returns[var_percentile_index_monthly]

# Step 3: Print the monthly VaR result
print(f"Monte Carlo Monthly VaR (95% confidence) for 1 month: {monte_carlo_monthly_var:.4%}")


Monte Carlo Monthly VaR (95% confidence) for 1 month: 4.6317%


This means that using the Monte Carlo simulation method our 5% VaR for the portfolio is 4.6317%. Or that there is a 5% chance of losing at least 231,585 in a given month.

Comparing between the three different methods: 

Daily VaR values - 1.03% (parametric), 1.0973% (historic), 1.0357% (Monte Carlo). 

Monthly VaR values - 3.36% (parametric), 2.2901% (historic), 4.6317% (Monte Carlo). 

VaR has many benefits and pitfalls. It is extremely easy to compute and understand, explaining it to someone without a strong finanace background is very feasible. There are several methods so you can compare the potential outcomes and make a better decision.
However there are some problems with finding / using VaR. In this specific project there are some potential reasons behind the gaps. 

1) Distrubition assumptions:  Parametric method assumes normal distribution, which may not fully capture extreme market moves. Monte Carlo can model non-normal distributions (IE fat tails) so it can produce higher VaR values, especially in the monthly time frame. 

2) Historical data period: the period chosen could've included market downturn or extreme volatility events. Historical VaR directly uses observed data, and if this data does not represent extreme events (e.g., a period of relative calm), the VaR estimate will be lower than the one based on parametric or Monte Carlo simulations. 

3) Time horizon scaling: The scaling of VaR from daily to monthly in the parametric method assumes that volatility is constant over time, which may not hold true in real markets. Monte Carlo simulations model this more flexibly and may produce more extreme estimates of future risk. 

4) Simulation Number: If Monte Carlo simulation uses a large number of trials (e.g., 10,000+ simulations), it might catch extreme events better, leading to a higher VaR estimate, especially when compared to historical data that might not include such events.